In [47]:
import pandas as pd
import numpy as np
import matplotlib as mpl
mpl.rc('font', family='NanumGothic')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [48]:
df_member = pd.read_csv('member_data.csv')
df_product = pd.concat([pd.read_csv('product_data_A.csv'),
                         pd.read_csv('product_data_B.csv'),
                         pd.read_csv('product_data_C.csv'),
                         pd.read_csv('product_data_D.csv')])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [49]:
display(df_member.head(3))
display(df_member['모바일알람여부'].unique())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,회원번호,조합원상태,주소,성별,연령,배송서비스신청여부,모바일알람여부,Gold_member
0,1000012800,정상회원,수지구-풍덕천동,여,58.0,미신청,.,normal
1,1000103744,정상회원,처인구-고림동,여,34.0,미신청,.,normal
2,1000122304,정상회원,광주-기타,여,38.0,미신청,.,normal


array(['.', '수신'], dtype=object)

In [50]:
display(df_product.head(3))
display(df_product.info())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,공급일자,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,구매수량,구매금액,구매매장,반품_원거래일자,구매시각
0,2022-01-01,220954944,정상회원,반찬,두부/유부,두부,두부(1모:420g),1.0,3829,매장A,NaN,10:32
1,2022-01-01,89023040,정상회원,채소,열매채소,단호박/손질,단호박(깍뚝썰기/300g),1.0,4977,매장A,NaN,10:41
2,2022-01-01,354941440,정상회원,축산물,알,유정란,유정란/매장용(10알/국내산),1.0,7083,매장A,NaN,11:00


<class 'pandas.core.frame.DataFrame'>
Index: 294645 entries, 0 to 55107
Data columns (total 12 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   공급일자      294645 non-null  object 
 1   회원번호      294645 non-null  int64  
 2   조합원상태     294645 non-null  object 
 3   물품대분류     294645 non-null  object 
 4   물품중분류     294645 non-null  object 
 5   물품소분류     294645 non-null  object 
 6   물품명       294645 non-null  object 
 7   구매수량      294645 non-null  float64
 8   구매금액      294645 non-null  int64  
 9   구매매장      294645 non-null  object 
 10  반품_원거래일자  4960 non-null    object 
 11  구매시각      294645 non-null  object 
dtypes: float64(1), int64(2), object(9)
memory usage: 29.2+ MB


None

In [51]:
#고객별 구매특성 pivot table 생성 : 회원번호별 횟수&금액
p1 = df_product.pivot_table(index='회원번호', values=['구매금액','구매수량'],aggfunc='sum').reset_index()
p1.head(3)
#p1.to_excel('고객별_구매특성(횟수&금액).xlsx', sheet_name='Sheet1')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,회원번호,구매금액,구매수량
0,18613824,21059,1.0
1,18764160,-88827,7.0
2,18792000,33311,1.0


In [52]:
import datetime

df_product['공급일자'] = pd.to_datetime(df_product['공급일자'])

df_product['구매시각'] = df_product['구매시각'].apply(lambda x: datetime.datetime.strptime(x, '%H:%M'))
df_product['구매시각'] = df_product['구매시각'].dt.time

df_product[['공급일자','구매시각']].head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,공급일자,구매시각
0,2022-01-01,10:32:00
1,2022-01-01,10:41:00
2,2022-01-01,11:00:00
3,2022-01-01,11:28:00
4,2022-01-01,11:59:00


## Apriori (장바구니 연관분석)

In [53]:
# 데이터 병합
df_product_member = pd.merge(df_product, df_member, on='회원번호')
df_product_member.head(3)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,공급일자,회원번호,조합원상태_x,물품대분류,물품중분류,물품소분류,물품명,구매수량,구매금액,구매매장,반품_원거래일자,구매시각,조합원상태_y,주소,성별,연령,배송서비스신청여부,모바일알람여부,Gold_member
0,2022-01-01,220954944,정상회원,반찬,두부/유부,두부,두부(1모:420g),1.0,3829,매장A,NaN,10:32:00,정상회원,수지구-동천동,남,51.0,미신청,수신,normal
1,2022-01-01,89023040,정상회원,채소,열매채소,단호박/손질,단호박(깍뚝썰기/300g),1.0,4977,매장A,NaN,10:41:00,정상회원,분당구-구미동,여,48.0,신청,.,normal
2,2022-01-01,354941440,정상회원,축산물,알,유정란,유정란/매장용(10알/국내산),1.0,7083,매장A,NaN,11:00:00,정상회원,수지구-동천동,여,50.0,미신청,.,normal


In [54]:
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# 분석에 사용할 컬럼: '물품대분류'
analysis_column = '물품대분류'

# '회원번호'별 구매 기록 생성
transactions = df_product_member.groupby('회원번호')[analysis_column].apply(list).values.tolist()

# TransactionEncoder를 사용하여 데이터 변환
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

# Apriori 알고리즘 적용
frequent_itemsets = apriori(df_encoded, min_support=0.01, use_colnames=True)

display(frequent_itemsets)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets
0,0.633665,(간식)
1,0.137862,(건강)
2,0.265599,(과실)
3,0.021549,(기타)
4,0.596019,(반찬)
...,...,...
9739,0.011597,"(과실, 양념/가루, 음료, 채소, 건강, 간식, 잡곡, 주곡, 생활용품, 축산물,..."
9740,0.012549,"(양념/가루, 반찬, 음료, 채소, 건강, 간식, 잡곡, 주곡, 생활용품, 축산물,..."
9741,0.020424,"(과실, 양념/가루, 반찬, 음료, 채소, 간식, 잡곡, 주곡, 생활용품, 축산물,..."
9742,0.011597,"(과실, 양념/가루, 반찬, 음료, 채소, 건강, 잡곡, 주곡, 생활용품, 축산물,..."


In [55]:
!pip install mlxtend --upgrade
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [56]:

# Assuming df_product_member is already loaded

# 1. 연관 규칙 분석을 위한 데이터 준비 : 분석 변수 선택('물품대분류')
analysis_column = '물품대분류'

# 각 고객이 구매한 물품 리스트 생성 (각 거래는 고객이 구매한 물품들의 list로 표현)
transactions = df_product_member.groupby('회원번호')[analysis_column].apply(list).values.tolist()


# 2. TransactionEncoder를 사용해 거래 데이터를 인코딩
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

# 3. apriori 알고리즘을 적용해 자주 발생한 항목집합 찾기
frequent_itemsets = apriori(df_encoded, min_support=0.01, use_colnames=True)
display(frequent_itemsets)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets
0,0.633665,(간식)
1,0.137862,(건강)
2,0.265599,(과실)
3,0.021549,(기타)
4,0.596019,(반찬)
...,...,...
9739,0.011597,"(과실, 양념/가루, 음료, 채소, 건강, 간식, 잡곡, 주곡, 생활용품, 축산물,..."
9740,0.012549,"(양념/가루, 반찬, 음료, 채소, 건강, 간식, 잡곡, 주곡, 생활용품, 축산물,..."
9741,0.020424,"(과실, 양념/가루, 반찬, 음료, 채소, 간식, 잡곡, 주곡, 생활용품, 축산물,..."
9742,0.011597,"(과실, 양념/가루, 반찬, 음료, 채소, 건강, 잡곡, 주곡, 생활용품, 축산물,..."


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [57]:
# 3-1.frequent_itemsets 결과 해석

# support 값 분석
print('** 연관분석 통계 **')
display(frequent_itemsets['support'].describe())

print('\n')

# 높은 support를 가진 항목집합 식별 => 상위 75% 이상의 값
high_support_itemsets = frequent_itemsets[frequent_itemsets['support'] > frequent_itemsets['support'].quantile(0.75)]
print('** support(지지도) 상위 75%이상 결과 **')
display(high_support_itemsets)

print('\n')

# 가장 높은 support를 가진 항목집합 탐색
print('** support(지지도)값이 가장 높은 10개 항목 **')
display(frequent_itemsets.sort_values('support', ascending=False).head(10))


# 결과 해석:
# - 'support' 열은 데이터셋에서 각 항목집합의 빈도를 나타냄.
# - 높은 support 값은 해당 항목집합이 거래에서 더 자주 나타남을 의미.
# - 높은 support를 가진 항목집합을 분석함으로써 자주 함께 발생하는 항목을 식별 가능.
# - 'itemsets' 열은 각 항목집합에 속한 특정 항목들을 보여줌.
# - 추가 분석으로는 support 값을 시각화하는 방법(예: support 값의 막대 그래프)과 가장 빈번한 항목집합에 대한 상세 탐색이 포함될 수 있음.

** 연관분석 통계 **


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support
count,9744.000000
mean,0.043636
std,0.044341
min,0.010039
25%,0.018434
50%,0.030463
75%,0.052012
max,0.651579




** support(지지도) 상위 75%이상 결과 **


,support,itemsets
0,0.633665,(간식)
1,0.137862,(건강)
2,0.265599,(과실)
4,0.596019,(반찬)
5,0.304024,(생활용품)
...,...,...
9036,0.055041,"(과실, 양념/가루, 반찬, 음료, 채소, 간식, 잡곡, 축산물, 수산)"
9039,0.061965,"(과실, 양념/가루, 반찬, 음료, 채소, 간식, 주곡, 축산물, 수산)"
9090,0.061099,"(양념/가루, 반찬, 음료, 채소, 간식, 생활용품, 축산물, 서류, 수산)"
9109,0.053397,"(양념/가루, 반찬, 음료, 채소, 간식, 잡곡, 생활용품, 축산물, 수산)"




** support(지지도)값이 가장 높은 10개 항목 **


,support,itemsets
13,0.651579,(채소)
0,0.633665,(간식)
14,0.629424,(축산물)
4,0.596019,(반찬)
114,0.501341,"(채소, 축산물)"
68,0.489572,"(반찬, 채소)"
7,0.488879,(수산)
27,0.486196,"(간식, 채소)"
28,0.477196,"(간식, 축산물)"
69,0.467590,"(반찬, 축산물)"


In [58]:
# 4.연관규칙 생성
rules = association_rules(frequent_itemsets,
                          metric='confidence',
                          min_threshold=0.7,
                          num_itemsets=len(df_encoded))

display(rules.head(10))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(건강),(간식),0.137862,0.633665,0.112419,0.815443,1.286867,1.0,0.025060,1.984937,0.258565,0.170562,0.496206,0.496427
1,(과실),(간식),0.265599,0.633665,0.229424,0.863799,1.363180,1.0,0.061123,2.689670,0.362773,0.342506,0.628207,0.612929
2,(기타),(간식),0.021549,0.633665,0.018087,0.839357,1.324607,1.0,0.004432,2.280435,0.250456,0.028389,0.561487,0.433951
3,(반찬),(간식),0.596019,0.633665,0.459974,0.771744,1.217905,1.0,0.082298,1.604929,0.442887,0.597594,0.376920,0.748819
4,(간식),(반찬),0.633665,0.596019,0.459974,0.725895,1.217905,1.0,0.082298,1.473816,0.488400,0.597594,0.321489,0.748819
5,(생활용품),(간식),0.304024,0.633665,0.246993,0.812411,1.282083,1.0,0.054343,1.952860,0.316130,0.357599,0.487930,0.601098
6,(서류),(간식),0.180788,0.633665,0.158806,0.878411,1.386238,1.0,0.044247,3.012888,0.340111,0.242212,0.668093,0.564513
7,(수산),(간식),0.488879,0.633665,0.388923,0.795539,1.255457,1.0,0.079137,1.791711,0.398100,0.530140,0.441874,0.704653
8,(양념/가루),(간식),0.427607,0.633665,0.344007,0.804493,1.269587,1.0,0.073047,1.873769,0.370973,0.479609,0.466316,0.673689
9,(음료),(간식),0.384076,0.633665,0.311727,0.811627,1.280845,1.0,0.068351,1.944730,0.355995,0.441530,0.485790,0.651784


In [ ]:
"""
# 4-1.'rules' 해석 결과코드(4.연관분석 규칙 생성 코드 해석)

# 연관분석 규칙 생성 과정 출력
for index, row in rules.iterrows():
    antecedents = list(row['antecedents'])
    consequents = list(row['consequents'])
    support = row['support']
    confidence = row['confidence']
    lift = row['lift']

    print(f"규칙 {index + 1}:")
    print(f"  {antecedents} => {consequents}")
    print(f"  지지도: {support:.4f}")
    print(f"  신뢰도: {confidence:.4f}")
    print(f"  향상도: {lift:.4f}")
    print("-" * 20)
  """

In [64]:
# 4-1.'rules' 해석 결과코드(4.연관분석 규칙 생성 코드 해석)

# 연관분석 규칙 생성 과정 출력 및 결과 저장
output_file = "원마트_장바구니연관분석_규칙생성_과정.txt"

# 파일 열기 (쓰기 모드)
with open(output_file, "w", encoding="utf-8") as file:
    for index, row in rules.iterrows():
        antecedents = list(row['antecedents'])
        consequents = list(row['consequents'])
        support = row['support']
        confidence = row['confidence']
        lift = row['lift']

        # 출력 형식
        rule_output = (
            f"규칙 {index + 1}:\n"
            f"  {antecedents} => {consequents}\n"
            f"  지지도: {support:.4f}\n"
            f"  신뢰도: {confidence:.4f}\n"
            f"  향상도: {lift:.4f}\n"
            + "-" * 20 + "\n"
        )

        # 화면 출력
        print(rule_output)

        # 파일에 쓰기
        file.write(rule_output)

print(f"규칙 분석 결과가 '{output_file}'에 저장되었습니다.")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.

규칙 250465:
  ['과실', '음료', '건강', '잡곡', '주곡', '생활용품', '서류', '수산'] => ['반찬', '채소', '축산물', '양념/가루']
  지지도: 0.0116
  신뢰도: 0.9853
  향상도: 3.3614
--------------------

규칙 250466:
  ['과실', '음료', '건강', '잡곡', '주곡', '축산물', '서류', '수산'] => ['반찬', '채소', '생활용품', '양념/가루']
  지지도: 0.0116
  신뢰도: 0.8874
  향상도: 5.2184
--------------------

규칙 250467:
  ['과실', '음료', '건강', '잡곡', '생활용품', '축산물', '서류', '수산'] => ['반찬', '채소', '주곡', '양념/가루']
  지지도: 0.0116
  신뢰도: 0.7322
  향상도: 5.9795
--------------------

규칙 250468:
  ['과실', '채소', '건강', '잡곡', '주곡', '생활용품', '축산물', '서류'] => ['반찬', '음료', '수산', '양념/가루']
  지지도: 0.0116
  신뢰도: 0.8645
  향상도: 4.7957
--------------------

규칙 250469:
  ['과실', '채소', '건강', '잡곡', '주곡', '생활용품', '서류', '수산'] => ['반찬', '음료', '축산물', '양념/가루']
  지지도: 0.0116
  신뢰도: 0.8933
  향상도: 4.4609
--------------------

규칙 250470:
  ['과실', '채소', '건강', '잡곡', '주곡', '축산물', '서류', '수산'] => ['반찬', '음료', '생활용품', '양념/가루']
  지지도: 0.0116
  신뢰도: 0.7657
  향상도: 5.8946
--------------------

규칙 

In [67]:
# 용량이 너무 커서 20mb씩 나눠서 저장하는 코드
"""
# 연관분석 규칙 생성 과정 출력 및 결과 저장
import os

output_dir = "association_rules_output"
os.makedirs(output_dir, exist_ok=True)  # 출력 디렉토리 생성

chunk_size = 20 * 1024 * 1024  # 20MB
current_chunk = 1
current_file_size = 0

output_file = os.path.join(output_dir, f"원마트_장바구니연관분석_규칙생성_상세과정{current_chunk}.txt")
file = open(output_file, "w", encoding="utf-8")

for index, row in rules.iterrows():
    antecedents = list(row['antecedents'])
    consequents = list(row['consequents'])
    support = row['support']
    confidence = row['confidence']
    lift = row['lift']

    # 출력 형식
    rule_output = (
        f"규칙 {index + 1}:\n"
        f"  {antecedents} => {consequents}\n"
        f"  지지도: {support:.4f}\n"
        f"  신뢰도: {confidence:.4f}\n"
        f"  향상도: {lift:.4f}\n"
        + "-" * 20 + "\n"
    )

    # 파일에 쓰기
    file.write(rule_output)
    current_file_size += len(rule_output.encode('utf-8'))  # UTF-8 기준 바이트 크기 계산

    # 파일 크기 초과 시 새 파일 열기
    if current_file_size >= chunk_size:
        file.close()
        current_chunk += 1
        current_file_size = 0
        output_file = os.path.join(output_dir, f"원마트_장바구니연관분석_규칙생성_상세과정{current_chunk}.txt")
        file = open(output_file, "w", encoding="utf-8")

file.close()
print(f"규칙 분석 결과가 '{output_dir}' 디렉토리에 나눠 저장되었습니다.")
"""

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


규칙 분석 결과가 'association_rules_output' 디렉토리에 나눠 저장되었습니다.


In [65]:
# 규칙을 confidence(신뢰도) 값으로 내림차순 정렬.
rules_sorted = rules.sort_values(by=['confidence'], ascending=False)

print("상위 10개 규칙:")
display(rules_sorted.head(10))

상위 10개 규칙:


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
223052,"(양념/가루, 반찬, 음료, 주류, 채소, 주곡, 생활용품, 수산)","(간식, 축산물)",0.010212,0.477196,0.010212,1.0,2.095575,1.0,0.005339,inf,0.528198,0.021400,1.0,0.510700
151183,"(과실, 건강, 간식, 잡곡, 주곡, 생활용품, 축산물)","(반찬, 채소)",0.018001,0.489572,0.018001,1.0,2.042602,1.0,0.009188,inf,0.519785,0.036769,1.0,0.518384
151180,"(과실, 채소, 건강, 간식, 잡곡, 주곡, 생활용품)","(반찬, 축산물)",0.018001,0.467590,0.018001,1.0,2.138627,1.0,0.009584,inf,0.542170,0.038497,1.0,0.519249
12928,"(주류, 축산물, 과실, 서류)",(채소),0.011770,0.651579,0.011770,1.0,1.534732,1.0,0.004101,inf,0.352570,0.018063,1.0,0.509032
151093,"(과실, 반찬, 음료, 채소, 건강, 간식, 주곡, 생활용품)",(축산물),0.024232,0.629424,0.024232,1.0,1.588753,1.0,0.008980,inf,0.379778,0.038499,1.0,0.519249
151096,"(과실, 반찬, 음료, 건강, 간식, 주곡, 생활용품, 축산물)",(채소),0.024232,0.651579,0.024232,1.0,1.534732,1.0,0.008443,inf,0.357073,0.037190,1.0,0.518595
78184,"(반찬, 건강, 잡곡, 주곡, 서류)","(채소, 축산물)",0.017828,0.501341,0.017828,1.0,1.994649,1.0,0.008890,inf,0.507710,0.035560,1.0,0.517780
78181,"(반찬, 채소, 건강, 잡곡, 주곡, 서류)",(축산물),0.017828,0.629424,0.017828,1.0,1.588753,1.0,0.006607,inf,0.377302,0.028324,1.0,0.514162
78182,"(반찬, 건강, 잡곡, 주곡, 축산물, 서류)",(채소),0.017828,0.651579,0.017828,1.0,1.534732,1.0,0.006212,inf,0.354745,0.027361,1.0,0.513680
151103,"(과실, 반찬, 음료, 건강, 간식, 주곡, 생활용품)","(채소, 축산물)",0.024232,0.501341,0.024232,1.0,1.994649,1.0,0.012083,inf,0.511042,0.048334,1.0,0.524167


In [66]:
# lift(향상도)가 1보다 큰 규칙을 필터링. (의미있는 연관성을 갖는 규칙)
significant_rules = rules[rules['lift'] > 1]

# 의미있는 규칙을 출력합니다.
print("의미있는 규칙:")
display(significant_rules)

의미있는 규칙:


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(건강),(간식),0.137862,0.633665,0.112419,0.815443,1.286867,1.0,0.025060,1.984937,0.258565,0.170562,0.496206,0.496427
1,(과실),(간식),0.265599,0.633665,0.229424,0.863799,1.363180,1.0,0.061123,2.689670,0.362773,0.342506,0.628207,0.612929
2,(기타),(간식),0.021549,0.633665,0.018087,0.839357,1.324607,1.0,0.004432,2.280435,0.250456,0.028389,0.561487,0.433951
3,(반찬),(간식),0.596019,0.633665,0.459974,0.771744,1.217905,1.0,0.082298,1.604929,0.442887,0.597594,0.376920,0.748819
4,(간식),(반찬),0.633665,0.596019,0.459974,0.725895,1.217905,1.0,0.082298,1.473816,0.488400,0.597594,0.321489,0.748819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251173,"(건강, 잡곡, 주곡, 생활용품, 축산물, 서류)","(과실, 양념/가루, 반찬, 음료, 채소, 간식, 수산)",0.014539,0.112938,0.011597,0.797619,7.062443,1.0,0.009955,4.383129,0.871071,0.100075,0.771852,0.450151
251174,"(건강, 잡곡, 주곡, 생활용품, 서류, 수산)","(과실, 양념/가루, 반찬, 음료, 채소, 간식, 축산물)",0.014106,0.119602,0.011597,0.822086,6.873518,1.0,0.009910,4.948444,0.866741,0.094968,0.797916,0.459523
251175,"(과실, 건강, 잡곡, 주곡, 서류)","(양념/가루, 반찬, 음료, 채소, 간식, 생활용품, 축산물, 수산)",0.015837,0.111986,0.011597,0.732240,6.538669,1.0,0.009823,3.316460,0.860695,0.099777,0.698474,0.417898
251176,"(음료, 건강, 잡곡, 주곡, 서류)","(과실, 양념/가루, 반찬, 채소, 간식, 생활용품, 축산물, 수산)",0.015318,0.097187,0.011597,0.757062,7.789718,1.0,0.010108,3.716229,0.885185,0.114923,0.730910,0.438193


### 4-1. 코드 해석(from. ChatGPT)

이 코드는 `association_rules` 함수를 사용해 연관 규칙을 생성하고 이를 화면에 출력하는 역할을 합니다. 각각의 파라미터가 의미하는 바는 아래와 같습니다.

---

1. **`rules = association_rules(...)`**  
   - `association_rules` 함수는 `frequent_itemsets` (빈발 항목집합)으로부터 연관 규칙을 생성합니다.  
   - 연관 규칙은 특정 항목 집합(antecedent)이 나타날 때, 다른 항목 집합(consequent)이 나타날 확률을 분석하는 규칙입니다.  

---

2. **파라미터 설명:**
   - **`frequent_itemsets`**:  
     빈발 항목집합 데이터가 포함된 DataFrame. 이 데이터는 `apriori` 함수의 출력 결과입니다.  
   
   - **`metric='confidence'`**:  
     연관 규칙을 생성할 때 평가 기준으로 `confidence`(신뢰도)를 사용합니다.  
     - `confidence`는 조건부 확률로, 선행 항목집합(antecedent)이 발생했을 때 후행 항목집합(consequent)이 발생할 확률을 나타냅니다.  

   - **`min_threshold=0.7`**:  
     `confidence`가 최소 0.7 이상인 규칙만 반환합니다.  
     - 즉, 신뢰도가 70% 이상인 연관 규칙만 선택합니다.  

   - **`num_itemsets=len(df_encoded)`** (잘못된 부분):  
     이 인수는 `association_rules` 함수에서 지원되지 않습니다. 따라서 이 부분은 제거해야 합니다.

---

3. **`display(rules)`**  
   생성된 연관 규칙을 DataFrame 형식으로 출력합니다.  
   - 이 DataFrame에는 `antecedents`, `consequents`, `support`, `confidence`, `lift` 등 연관 규칙의 세부 정보가 포함됩니다.

---

### 예상 출력 결과:
`rules`는 연관 규칙을 포함한 DataFrame입니다. 주요 열의 의미는 아래와 같습니다:
- **`antecedents`**: 선행 항목집합.  
- **`consequents`**: 후행 항목집합.  
- **`support`**: 데이터셋에서 이 규칙이 등장한 비율.  
- **`confidence`**: 선행 항목이 발생했을 때 후행 항목이 발생할 확률.  
- **`lift`**: 연관성의 강도를 나타내는 지표. 1보다 크면 두 항목 사이에 양의 상관관계가 있음을 의미.

---